In [59]:
from dwavebinarycsp import stitch, irreducible_components, Constraint
import operator

def squeeze_same_results(results):
    squeezed_results = [results[0]]
    for item in results[1:]:
        for squeezed_item in squeezed_results:
            if item['results'] == squeezed_item['results']:
                if item['min_energy'] < squeezed_item['min_energy']:
                    squeezed_item['min_energy'] = item['min_energy']
                    squeezed_item['occurrences'] = item['occurrences']
                elif item['min_energy'] == squeezed_item['min_energy']:
                    squeezed_item['occurrences'] += item['occurrences']
                break
        else:
            squeezed_results.append(item)
    return squeezed_results


def parse_result(response):
    parsed_response = {}
    for datum in response.data(['sample', 'energy', 'num_occurrences']):
        key = (tuple(dict(datum.sample).items()), float(datum.energy))
        if key in parsed_response:
            parsed_response[key] = (datum.sample, parsed_response[key][1] + datum.num_occurrences)
        else:
            parsed_response[key] = (datum.sample, datum.num_occurrences)

    num_runs = sum([parsed_response[key][1] for key in parsed_response])
    results = []
    for key in parsed_response:
        results.append({
            'results': parsed_response[key][0],
            'min_energy': key[1],
            'occurrences': parsed_response[key][1] / num_runs * 100
        })
    return squeeze_same_results(results)


def get_response(response, embedding=None, qubits=None):
    results = parse_result(response)
    if embedding is not None:
        embedding_results = []
        for item in results:
            embedding_item = {
                'results': {},
                'min_energy': item['min_energy'],
                'occurrences': item['occurrences']
            }
            for A in embedding:
                embedding_item['results'][A] = sum([item['results'][qubit] for qubit in embedding[A]])
                embedding_item['results'][A] /= len(embedding[A])
                embedding_item['results'][A] = round(embedding_item['results'][A], 2)
            embedding_results.append(embedding_item)
        results = squeeze_same_results(embedding_results)
    if qubits is not None:
        new_results = []
        for item in results:
            new_results.append({
                'results': {key: item['results'][key] for key in qubits},
                'min_energy': item['min_energy'],
                'occurrences': item['occurrences']
            })
        results = new_results
    return squeeze_same_results(results)


def print_response(response, embedding=None, qubits=None):
    results = get_response(response, embedding=embedding, qubits=qubits)
    for item in results:
        print(item['results'], "Minimum energy: ", item['min_energy'],
              f"Occurrences: {item['occurrences']:.2f}%")


# TODO Need refactoring
def get_response_only_minimal(response, embedding=None, qubits=None):
    results = get_response(response, embedding=embedding, qubits=qubits)
    min_energy = min(map(lambda x: x['min_energy'], results))
    results = list(filter(lambda x: x['min_energy'] == min_energy, results))
    return squeeze_same_results(results)


def print_response_only_minimal(response, embedding=None, qubits=None):
    results_only_minimal = get_response_only_minimal(response, embedding=embedding,
                                                     qubits=qubits)
    for item in results_only_minimal:
        print(item['results'], "Minimum energy: ", item['min_energy'],
              f"Occurrences: {item['occurrences']:.2f}%")
    print()
    total = sum([item['occurrences'] for item in results_only_minimal])
    print(f"Total: {total:.2f}%")


import dwavebinarycsp
import dimod
from collections import Counter
from dwave.system.composites import FixedEmbeddingComposite
from dwave.system.samplers import DWaveSampler

def Qubo(weights, v_all, qbits_Num):
    #Q = {('x0','x0'): -1, ('x0','x1'): 2, ('x1', 'x0'): 0, ('x1','x1'): -1}
    Q = dict()
    for i in range(qbits_Num):
        for j in range(qbits_Num):
            Q[(v_all[i], v_all[j])] = weights[i][j]
    return Q


class Boltzman:
    weights = [] #w[i][j]
    q_in = {} #List explicitly specifying each allowed configuration as a tuple q_in = {(0, 0, 1)}
    number_in = 100
    v_in = [] # v_in = ['v0', 'v1'] or v_in = [0, 1]
    v_all = []
    number_out = 100
    embedding = ()
    qbits_Num = 200 # = number_out + number_in
    def run(self):
        # Create a binary constraint satisfaction problem
        csp = dwavebinarycsp.ConstraintSatisfactionProblem(dwavebinarycsp.BINARY)
        csp.add_constraint(self.q_in, self.v_in)
        bqm =  dwavebinarycsp.stitch(csp, min_classical_gap=2.0, max_graph_size=8) #csp to BinaryQuadraticModel, penaltymodel required(if there are mpre then 2 variables)
        QUBO_CSP = dimod.BinaryQuadraticModel.to_qubo(bqm)
        QUBO_weights = Qubo(self.weights, self.v_all, self.qbits_Num)
        
        #bqm_w = dimod.BinaryQuadraticModel.from_qubo(QUBO_weights)

        QUBO_sum_Counter = (Counter(QUBO_weights)) + Counter(QUBO_CSP[0]))
        QUBO_sum = dict(QUBO_sum_Counter)
        samp = DWaveSampler()
        sampler_embedded = FixedEmbeddingComposite(samp, self.embedding)
        response_BM = sampler_embedded.sample_qubo(QUBO_sum, num_reads= 5000)
        Solution = get_response_only_minimal(response_BM)
    
        return Solution

            
        

SyntaxError: invalid syntax (<ipython-input-59-1feb256d1949>, line 128)

In [60]:
from dwave.cloud import Client
client = Client.from_config(token = '')
client.get_solvers()
r = Boltzman()
r.weights = [[0, 3, 1], [0, 0, 2], [0, 0, 0]]
r.q_in = {(0, 1)}
r.v_in = ['0', '1']
r.v_all = ['0', '1', '2']
r.embedding ={'0': {0}, '1':{4}, '2':{3,7}}
r.qbits_Num = 3
r.run()

{('0', '0'): 0, ('0', '1'): 3, ('0', '2'): 1, ('1', '0'): 0, ('1', '1'): 0, ('1', '2'): 2, ('2', '0'): 0, ('2', '1'): 0, ('2', '2'): 0} {('0', '0'): 2, ('1', '1'): -2}


SolverFailureError: Problem not accepted because user kirill.parshukov.14@gmail.com has no remaining quota in project DEV